In [49]:
import pandas as pd
import matplotlib as plt
import numpy as np
import datetime

In [2]:
pd.set_option('display.max_columns',999)

In [3]:
def file_head(file):
    with open(file) as csv:
        tabular = []
        for x in range(10):
            tabular.append(csv.readline().strip().split(','))
        return pd.DataFrame(tabular[1:],columns=tabular[0])

In [4]:
!cd data & dir


 Volume in drive H has no label.
 Volume Serial Number is 1425-AB82

 Directory of H:\Kaggle\elo-merchant-category-recommendation\data

12/18/2018  03:22 PM    <DIR>          .
12/18/2018  03:22 PM    <DIR>          ..
12/18/2018  03:08 PM            16,917 Data_Dictionary.xlsx
11/23/2018  09:08 PM     2,845,920,484 historical_transactions.csv
11/23/2018  09:08 PM        50,040,976 merchants.csv
11/23/2018  09:08 PM       190,246,145 new_merchant_transactions.csv
11/23/2018  09:08 PM         2,225,229 sample_submission.csv
11/23/2018  09:08 PM         3,708,740 test.csv
11/23/2018  09:08 PM         8,383,651 train.csv
               7 File(s)  3,100,542,142 bytes
               2 Dir(s)   8,715,476,992 bytes free


merchant_id - Unique merchant identifier<br>
merchant_group_id - Merchant group (anonymized )<br>
merchant_category_id - Unique identifier for merchant category (anonymized )<br>
subsector_id - Merchant category group (anonymized )<br>
numerical_1 - anonymized measure <br>
numerical_2 - anonymized measure <br>
category_1 - anonymized category <br>
most_recent_sales_range - Range of revenue (monetary units) in last active month --> A > B > C > D > E <br>
most_recent_purchases_range - Range of quantity of transactions in last active month --> A > B > C > D > E <br>
avg_sales_lag3 - Monthly average of revenue in last 3 months divided by revenue in last active month <br>
avg_purchases_lag3 - Monthly average of transactions in last 3 months divided by transactions in last active month <br>
active_months_lag3 - Quantity of active months within last 3 months <br>
avg_sales_lag6 - Monthly average of revenue in last 6 months divided by revenue in last active month <br>
avg_purchases_lag6 - Monthly average of transactions in last 6 months divided by transactions in last active month <br>
active_months_lag6 - Quantity of active months within last 6 months <br>
avg_sales_lag12 - Monthly average of revenue in last 12 months divided by revenue in last active month <br>
avg_purchases_lag12 - Monthly average of transactions in last 12 months divided by transactions in last active month <br>
active_months_lag12 - Quantity of active months within last 12 months <br>
category_4 - anonymized category <br>
city_id	City - identifier (anonymized ) <br>
state_id - State identifier (anonymized ) <br>
category_2 - anonymized category<br>

In [5]:
dtps = {'city_id':np.int8,'installments':np.int8,'merchant_category_id':np.int16,'month_lag':np.int8,'state_id':np.int16,'subsector_id':np.int16}
hist_trans = pd.read_csv('data\\historical_transactions.csv',dtype=dtps,parse_dates=[10])
merchants = pd.read_csv('data\\merchants.csv')
merch_trans = pd.read_csv('data\\new_merchant_transactions.csv')


In [50]:
def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days
    return df

In [54]:
train = read_data('data/train.csv')
test = read_data('data/test.csv')

target = train['target']
del train['target']

In [10]:
merch_trans.authorized_flag.replace({'N':2,'Y':1},inplace=True)

In [11]:
merch_trans['makes_unauthorized_trans'] = (merch_trans.authorized_flag == 2)

In [12]:
merch_trans.city_id.replace({-1:404},inplace=True)

In [13]:
merch_trans['unidentified_city'] = merch_trans.city_id == 404

In [14]:
merch_trans.category_1.replace({'Y':1,'N':2},inplace=True)

In [15]:
merch_trans['cat1_no'] = merch_trans.category_1 == 2

In [16]:
merch_trans.installments.replace({-1:404,0:12},inplace=True)

In [17]:
merch_trans['No_installments'] = merch_trans.installments == 404

In [18]:
merch_trans.category_3.fillna(4,inplace=True)

In [19]:
merch_trans.category_3.replace({'A':1,'B':2,'C':3},inplace=True)

In [20]:
merch_trans.merchant_category_id.replace({-1:999},inplace=True)

In [21]:
merch_trans['no_purchase'] = merch_trans.purchase_amount == -1

In [22]:
merch_trans['purchase_bins'] = pd.cut(merch_trans.purchase_amount,16,labels=range(1,17))

In [23]:
merch_trans['purchase_less_than_zero'] = merch_trans.purchase_amount < 0

In [24]:
merch_trans.category_2.fillna(6,inplace=True)

In [25]:
merch_trans['No_cat2'] = merch_trans.category_2 == 6

In [26]:
merch_trans.state_id.replace({-1:25},inplace=True)

In [27]:
merch_trans['No_state_id'] = merch_trans.state_id==25

In [28]:
merch_trans.subsector_id.replace({-1:42},inplace=True)

In [29]:
merch_trans['No_subsector_id'] = merch_trans.subsector_id==42

In [30]:
merch_trans.purchase_date = merch_trans.purchase_date.astype(np.datetime64)

In [31]:
kwords = ['year','dayofweek','month','hour','is_month_end','is_month_start']
for x in kwords:
    merch_trans['pdate_'+x] = getattr(merch_trans.purchase_date.dt,x)

In [32]:
merch_trans.pdate_dayofweek += 1

In [33]:
merch_trans.pdate_month += 1

In [34]:
merch_trans['Is_a_weekend'] = merch_trans.pdate_dayofweek > 5

In [35]:
num_of_merchant = merch_trans.groupby(['city_id',])['merchant_id'].nunique()

In [36]:
def bfunc(x):
    for pair in [(10000,1),(7000,2),(4000,3),(1000,4),(700,5),(300,6),(0,7)]:
        if x > pair[0]:
            return pair[1]
        
reps = {x:bfunc(y) for x,y in num_of_merchant.iteritems()}

In [37]:
reps[404] = 8

In [39]:
hist_trans

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37
5,Y,C_ID_4e6213e9bc,77,N,0,A,80,M_ID_50af771f8d,0,-0.734887,2018-02-24 08:45:05,1.0,9,37
6,Y,C_ID_4e6213e9bc,88,N,0,A,278,M_ID_5e8220e564,-11,-0.716855,2017-03-21 00:10:51,1.0,16,37
7,Y,C_ID_4e6213e9bc,3,N,0,A,80,M_ID_9d41786a50,-3,-0.657049,2017-11-18 20:05:55,1.0,16,37
8,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-8,-0.737967,2017-06-01 22:02:56,1.0,16,37
9,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_74ba14b5fc,-11,-0.715352,2017-03-16 15:41:22,1.0,16,37


In [40]:
hist_trans.installments.replace({-1:13,-25:25},inplace=True)

In [41]:
hist_trans.category_2.fillna(6,inplace=True)

In [42]:
hist_trans.category_3.fillna(4,inplace=True)

In [43]:
hist_trans.category_3.replace({'A':1,'B':2,'C':3},inplace=True)

In [45]:
cat1dm = pd.get_dummies(hist_trans.category_1,prefix='cat1')

In [46]:
aufdm = pd.get_dummies(hist_trans.authorized_flag,prefix='au')